In [109]:
import os
import pandas as pd
import numpy as np
import datetime
import dataset
from dotenv import load_dotenv
from pathlib import Path
import json
from tqdm import tqdm

In [110]:
def calculate_market_analysis(prompt, system_template):
    return f"{prompt} {system_template}"

In [111]:
def create_url() -> str:
    """
    Construye la URL de conexión a la base de datos PostgreSQL
    utilizando variables de entorno.
    """
    load_dotenv()

    user = os.getenv('POSTGRES_USER', 'postgres')
    password = os.getenv('POSTGRES_PASSWORD', 'postgres')
    host = os.getenv('POSTGRES_HOST', 'localhost')
    port = os.getenv('POSTGRES_PORT', 5432)
    db = os.getenv('POSTGRES_DB', 'postgres')

    return f'postgresql://{user}:{password}@{host}:{port}/{db}'

In [112]:
def drop_table_data():
    """Borra la tabla 'data' en la base de datos."""
    url = create_url()
    db = dataset.connect(url=url)
    table = db['data']
    table.drop()

In [113]:
def create_table_data():
    url = create_url()
    db = dataset.connect(url=url)
    table = db['data']
    records=[]

    for data in json.load(Path("data/companies.json").open('r', encoding='utf-8')):
        df = pd.json_normalize(
            data=json.load(Path("data/market_analysis_template.json").open('r', encoding='utf-8')), 
            record_path=["assigned_prompts"], 
            meta=["company_name", "company_code", "system_id", "system_name", "system_description", "system_template"]
        )

        df["company_name"] = df["company_name"].apply(lambda x: x.format(company_name=data["company_name"]))
        df["company_code"] = df["company_code"].apply(lambda x: x.format(company_code=data["company_code"]))
        
        df["prompt"] = df["prompt"].apply(lambda x: x.format(company_name=data["company_name"]))
        
        df = df.assign(market_analysis="", score_market_analysis=0, created_at=datetime.datetime.now(), updated_at=datetime.datetime.now(), status=0, processed=False)
        
        for record in tqdm(df.to_dict(orient="records"), desc=f"create records {data['company_code']}"):
            records.append(record)


    for record in tqdm(records, desc="insert records"):
        if not table.find_one(company_code=record["company_code"], prompt_id=record["prompt_id"]):
            table.insert(record)

In [114]:
def update_table_data():
    url = create_url()
    db = dataset.connect(url=url)
    table = db['data']

    for record in tqdm(table.find(processed=False, status=0), desc="update records"):
        record['market_analysis'] = calculate_market_analysis(prompt=record['prompt'], system_template=record['system_template'])
        record['status'] = 1
        # record['processed'] = True
        table.update(record, ['id'])

In [ ]:
drop_table_data()
create_table_data()
update_table_data()

create records IMD: 100%|██████████| 10/10 [00:00<00:00, 86659.17it/s]


insert records: 100%|██████████| 2390/2390 [00:10<00:00, 226.16it/s]
update records: 802it [00:02, 309.23it/s]